In [1]:
import os, sys
print(os.listdir('../../'))
print(os.getcwd())
sys.path.append("../")
sys.path.append("../src/network")
sys.path.append("../src")
print(os.getcwd())
import torch
import glob
import string
import os, sys
import cv2
import numpy as np
from network.gen_model.gen_model import GenModel_FC
from random import choices
import random
import matplotlib.pyplot as plt
from preproc import preproc_iam
from torchvision import transforms
from tokenizer import Tokenizer
import multiprocessing


['.git', '.gitignore', 'data', 'Groundtruth', 'README.md', 'src', 'src_supervised_htr', 'test_french.ipynb']
c:\Users\Lisa\Documents\GitHub\ws-siamese\src\data
c:\Users\Lisa\Documents\GitHub\ws-siamese\src\data


In [2]:
charset_base = string.ascii_lowercase + string.ascii_uppercase
max_text_length = 25
tokenizer = Tokenizer(chars=charset_base, max_text_length=max_text_length)

### Loading generator model

In [3]:
gen_model = GenModel_FC(tokenizer.maxlen, tokenizer.vocab_size, tokenizer.PAD).cuda()
gen_model.load_state_dict(torch.load('../network/gen_model/gen_model.model')) #load
gen_model.eval()

RuntimeError: Error(s) in loading state_dict for GenModel_FC:
	size mismatch for enc_image.model.features.0.weight: copying a param with shape torch.Size([64, 15, 3, 3]) from checkpoint, the shape in current model is torch.Size([64, 25, 3, 3]).

### global variables

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
input_size = (64, 216, 1)
num_style_imgs = 15 # num imgs for generator to extract style from
batch_size = 32
dataset = "iam"
dataset_path = os.path.join("..", "..", "data", dataset, "words")
max_word_len = 10

### Preprocessing data

In [ ]:
preproc_bool = False
if preproc_bool:
    path_from = os.path.join("..", "..", "..", "handwriting-recognition", "raw", dataset)
    os.makedirs(dataset_path, exist_ok=True)
    preproc_iam(path_from, dataset_path)

### Reading in the data

In [ ]:
def read_data(folder, max_word_len):
    """Get data paths and labels (with max_word_len) of images in folder."""
    dataset = {}
    wid_dict = {}
    lens = {}
    dataset = []
    wid_dict = {}

    text_file = os.path.join(folder, f"ground_truth_filtered.txt")

    with open(text_file, encoding='utf-8') as data_file:
        lines = data_file.read().splitlines()

    for line in lines:
        line_split = line.split(' ')
        wid = None

        if not set(string.digits).isdisjoint(set(line_split[-1])):
            wid = line_split[-1]
            line_split = line_split[:-1]
            
        if len(line_split) > 2:
            img_path, gt_label = line_split[0], ''.join(line_split[1:])
        else:
            img_path, gt_label = line_split[0], line_split[1]
            
        if len(gt_label) > max_word_len:
            continue

        if len(gt_label) in lens.keys():
            lens[len(gt_label)] += 1
        else:
            lens[len(gt_label)] = 1
        
        img_path = img_path.replace("/", "\\")

        if wid is not None:
            if wid not in wid_dict.keys():
                wid_dict[wid] = []
            
            wid_dict[wid].append((img_path, gt_label))

        # img_path = os.path.join(folder, partition, img_path)
        dataset.append((img_path, gt_label, wid))
    print(f"number of words: {len(dataset)}")
    print(f"number of wids: {len(wid_dict.keys())}")

    lens = dict(sorted(lens.items()))

    print(f"Number of words per word length")
    for key in lens.keys():
        print(key, lens[key])

    return dataset, wid_dict

In [ ]:
data, wids = read_data(dataset_path, max_word_len)

number of words: 31295
number of wids: 500
Number of words per word length
2 6677
3 8172
4 6304
5 4242
6 3109
7 2791


### Unique words in dataset

In [ ]:
unique_word_counts = []
all_words = []

word_len_dict = {}

for wid in wids.keys():
    words = list(set([wids[wid][idx][1] for idx in range(len(wids[wid]))]))
    all_words += words
    unique_word_counts.append(len(words))
unique_word_counts = sorted(unique_word_counts, reverse=True)
len(set(all_words))

for word in set(all_words):
    if len(word) not in word_len_dict.keys():
        word_len_dict[len(word)] = []
    word_len_dict[len(word)].append(word)


5% of unique words to validation
5% of unique words to test

In [ ]:
oov_test_words = []
oov_valid_words = []

random.seed(42)
for word_len in word_len_dict.keys():
    words = word_len_dict[word_len]
    print(word_len, len(words)// 10)
    oov_test_valid_words = random.sample(words, len(words) // 10)
    oov_test_words += oov_test_valid_words[:len(oov_test_valid_words)//2]
    oov_valid_words += oov_test_valid_words[len(oov_test_valid_words)//2:]


print(len(oov_test_words), len(oov_valid_words),len(set(all_words)))

6 113
4 78
7 110
5 104
3 27
2 6
218 220 4419


### Generate validation, test, and training splits

First 80/20 train/test
then 80/20 train/valid


In [ ]:
oov_test_data = []
oov_valid_data = []

train_data = []
valid_data = []
test_data = []

for wid in wids.keys():
    wid_data = wids[wid]
    train_wid = []
    test_wid = []
    wid_valid = []

    for tup in wid_data:
        if tup[1] in oov_test_words:
            oov_test_data.append((tup[0], tup[1], wid))
        elif tup[1] in oov_valid_words:
            oov_valid_data.append((tup[0], tup[1], wid))
        else:
            train_wid.append((tup[0], tup[1], wid))
    
    split_test_idx = len(train_wid) // 5
    if split_test_idx > 0:
        wid_test = train_wid[:split_test_idx]
        train_wid = train_wid[split_test_idx:]
    
    split_valid_idx = len(train_wid) // 5
    if split_valid_idx > 1:
        wid_valid = train_wid[:split_valid_idx]
        train_wid = train_wid[split_valid_idx:]
    
    if len(wid_test) > 0:
        test_data += wid_test
    
    if len(wid_valid) > 0:
        valid_data += wid_valid
    
    train_data += train_wid

print(len(train_data), len(valid_data), len(test_data))
print(train_data[0],  valid_data[0], test_data[0])

18745 4451 5550
('..\\..\\data\\iam\\words\\a03\\a03-034\\a03-034-01-03.png', 'booklet', '049') ('..\\..\\data\\iam\\words\\a03\\a03-034\\a03-034-00-04.png', 'are', '049') ('..\\..\\data\\iam\\words\\a03\\a03-034\\a03-034-00-00.png', 'Members', '049')


In [ ]:
character_freq_dict = {charset_base[idx] : 0 for idx in range(len(charset_base))}

for tup in train_data:
    mg_path, gt_label, wid = tup[0], tup[1], tup[2]
    for char in list(gt_label):
        character_freq_dict[char] += 1

total = sum(character_freq_dict.values())

for key in character_freq_dict.keys():
    character_freq_dict[key] /= total

print(character_freq_dict)
char_weights = [character_freq_dict[char] for char in charset_base]


{'a': 0.07946946670350925, 'b': 0.016137054434926777, 'c': 0.01769825918762089, 'd': 0.039748549323017406, 'e': 0.12830892511743575, 'f': 0.026554296767062724, 'g': 0.017463387676153633, 'h': 0.07033710969881182, 'i': 0.05732246476927328, 'j': 0.001091461729759602, 'k': 0.007667864050842774, 'l': 0.038643271621995025, 'm': 0.021566731141199225, 'n': 0.06417518651561205, 'o': 0.0803951367781155, 'p': 0.013539651837524178, 'q': 0.0005526388505111909, 'r': 0.053965183752417795, 's': 0.05815142304504007, 't': 0.09472229897761812, 'u': 0.023984526112185687, 'v': 0.008483006355346781, 'w': 0.025366123238463663, 'x': 0.0011605415860735009, 'y': 0.0206272450953302, 'z': 0.00030395136778115504, 'A': 0.001768444321635811, 'B': 0.0023210831721470018, 'C': 0.0013125172699640785, 'D': 0.000980933959657364, 'E': 0.0010638297872340426, 'F': 0.0013539651837524177, 'G': 0.0013539651837524177, 'H': 0.002680298424979276, 'I': 0.0016302846090080134, 'J': 0.0004973749654600719, 'K': 0.0003177673390439348, 

In [ ]:
transforms_ = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])

In [ ]:
def copy_style_imgs_wid(single_img, wid):
    """Obtain different same-style images based on the wid"""
    style_paths = wids[wid]
    copy_paths = random.choices(style_paths, k=15 - 1)

    final_img = [single_img[:]]

    for path in copy_paths:
        new_img = 255 - cv2.imread(path[0], cv2.IMREAD_GRAYSCALE) 
        new_img = transforms_(new_img)
        final_img = final_img + [new_img]

    final_img = np.stack(final_img, axis=0)

    return np.asarray(final_img, dtype="float32")

In [ ]:
def get_nonesense_label():
    word_len = random.randint(0, 10)

    if word_len == 0:
        return ''

    nonesense_label = random.choices(charset_base, weights=char_weights, k=word_len)
    return ''.join(nonesense_label)

In [ ]:
def random_insertion(text):
    idx = random.randint(0, len(text))
    char_in = random.choices(charset_base, weights=char_weights, k=1)
    return text[:idx] + char_in[0] + text[idx:]

In [ ]:
def random_deletion(text):
    idx = random.randint(0, len(text) - 1)
    return text[:idx] + text[idx + 1:]

In [ ]:
def random_swap(text):
    text = list(text)
    indices = random.choices([idx for idx in range(len(text))], k=2)
    temp = text[indices[0]]
    text[indices[0]] = text[indices[1]]
    text[indices[1]] = temp
    return ''.join(text)

In [ ]:
def random_replacement(text):
    idx = random.randint(0, len(text) - 1)
    new_char = random.choices(charset_base, weights=char_weights, k=1)
    text = list(text)
    text[idx] = new_char[0]
    return ''.join(text)

In [ ]:
def random_edits(text):
    edit_funcs = [random_insertion, random_deletion, random_swap, random_replacement]
    edits = random.choices(edit_funcs, k=3)
    text = edits[0](text)
    text = edits[1](text)
    return text

In [ ]:
def random_multi_insert(text):
    if len(text) > 2:
        num_pos = random.randint(1, 2)
    else:
        num_pos = 1
    for i in range(num_pos):
        idx = random.randint(0, len(text))
        char_in = text[-1] if idx == len(text) else text[idx]
        seq_in = char_in * random.randint(2, 5)
        text = text[:idx] + seq_in + text[idx:]
    return text[:25] if len(text) > 25 else text



In [ ]:
tbd = []
done = []
with open("../../data/iam/words/ground_truth_train_filtered.txt", 'w') as f:
    lines = f.read().splitlines()
    for line in lines:
        line = line.split(' ')[0]
        done.append(line)

for dat in train_data:
    dat_path, _, _ = dat
    if dat_path not in done:
        tbd.append(dat)

len(done)

65920

In [ ]:
# with open("../../data/iam/words/ground_truth_train_filtered.txt", 'w') as f:
#     lab = 1
#     for idx in range(0, len(done), 4):
#         pat = done[idx].split('.png')[0]
#         f.write(f"{pat}.png {pat}_sames.png 1\n")
#         f.write(f"{pat}.png {pat}_samed.png 1\n")
#         f.write(f"{pat}.png {pat}_nonesense.png 0\n")
#         f.write(f"{pat}.png {pat}_edits.png 0\n")



In [ ]:
between_diff_word = True
def generate_data(dataset, wid_ids):
    random.seed(42)
    # idx = 0
    pairs = []
    for tup in dataset:
        inputs = []
        labels = []
        img_path, gt_label, wid = tup[0], tup[1], tup[2]

        img = 255 - cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = transforms_(img)
        inputs.append(copy_style_imgs_wid(img, wid))
        labels.append(tokenizer.encode(gt_label))


        wid_diff = wid
        while wid_diff == wid:
            wid_diff = random.choice(wid_ids)

        inputs.append(copy_style_imgs_wid(img, wid_diff))
        labels.append(tokenizer.encode(gt_label))

        if random.random() <= 0.5:
            rnd_sample = random.choice(dataset)[0]
            while rnd_sample[1] == gt_label:
                rnd_sample = random.choice(dataset)[0]
            print(rnd_sample[1])
            nonesense_label = rnd_sample[1]
        else:
            nonesense_label = get_nonesense_label()

            if random.random() <= 0.25:
                nonesense_label = random_multi_insert(nonesense_label)
                
        inputs.append(copy_style_imgs_wid(img, wid))
        labels.append(tokenizer.encode(nonesense_label))
    
        edit_label = random_edits(gt_label)
        while edit_label == gt_label:
            edit_label = random_edits(gt_label)
        
        if random.random() <= 0.25:
            edit_label = random_multi_insert(edit_label)
        inputs.append(copy_style_imgs_wid(img, wid))
        labels.append(tokenizer.encode(edit_label))

        inputs = torch.from_numpy(np.array(inputs)).squeeze(2).cuda()

        labels = torch.from_numpy(np.array(labels)).cuda().long()
        labels = torch.nn.functional.one_hot(labels, 56).float()
        print(gt_label, nonesense_label, edit_label)

        synth_imgs = gen_model(inputs, labels).detach()
        # print(torch.min(synth_imgs), torch.max(synth_imgs))
        synth_imgs = ((synth_imgs * 0.5) + 0.5) * 255
        synth_imgs = synth_imgs.cpu().numpy()
        # print(torch.min(synth_imgs), torch.max(synth_imgs))

        new_img_path = img_path.split('.png')[0] # remove '.png
        # os.remove(new_img_path + '_same.png')
        
        cv2.imwrite(new_img_path + '_sames.png', synth_imgs[0][0])
        pairs += f"{img_path} {new_img_path}_sames.png 1 {gt_label} {gt_label} \n"
        # f.write(f"{img_path} {new_img_path}_sames.png 1 {gt_label} {gt_label} \n")

        cv2.imwrite(new_img_path + '_samed.png', synth_imgs[1][0])
        pairs += f"{img_path} {new_img_path}_samed.png 1 {gt_label} {gt_label} \n"
        # f.write(f"{img_path} {new_img_path}_samed.png 1 {gt_label} {gt_label} \n")

        cv2.imwrite(new_img_path + '_nonesense.png', synth_imgs[2][0])
        pairs += f"{img_path} {new_img_path}_nonesense.png 0 {gt_label} {nonesense_label}\n"
        # f.write(f"{img_path} {new_img_path}_nonesense.png 0 {gt_label} {nonesense_label}\n")

        cv2.imwrite(new_img_path + '_edits.png', synth_imgs[3][0])
        pairs += f"{img_path} {new_img_path}_edits.png 0 {gt_label} {edit_label}\n"
        # f.write(f"{img_path} {new_img_path}_edits.png 0 {gt_label} {edit_label}\n")


        # cv2.imwrite("./test.png", synth_imgs[0][0].cpu().numpy())
        # test = cv2.imread("./test.png")
        # test = torch.from_numpy(np.array(test)).cuda()
        # print(torch.max(test))
        # plt.imshow(img[0])
        # plt.show()
        # for img_ in synth_imgs:
        #     plt.imshow(img_[0].cpu())
        #     plt.show()
        # if i == 2:
        #     break
        # idx += 1





The otrnD Th
Record nBtarrn Recrvod
detail lnrtnelibo aetal
party nnwereipfg tyrpa
came tioewtFt cme
have eo fabe
timing stes timnxg
on eayaa n
budge nmee ubdee
credit osenii cedit
very Ti eryi
anyway tnot anyaI
times ei tsei
run st nru
put crnHh hsut
very hofmi ber
army oasCbnhje ary
have ao hae
oil paohh ipol
sprayed hyM spiaed
sprayed edioeiobE rpsahed
as h s
on trwo to
the iolofyltr he
pieces d piecses
correct gongevhh corre
pinned o wpinnd
was rowh aws
as tdtdThdHts is
very  vlyr
have yoleter haveo
was  fnas
was in w
Whigs pdfeei nWohigs
was hi awe
was riente wc
Naples doohats Nlles
effect rlebtc cbfeet
was momirb aos
was  whs
Naples aniyee baples
effect wi eoffct
give ynneP gse
was otieeaecah a
was nethieleea wa
as hloa 
on srfh noo
as rensu s
notice  noihe
When tDhi Wnlh
season ll sesao
was nolor easw
Lloyd oeofhpIo aLoyd
trouble aysdaeeeta uiotble
was e wgs
have utdeoe hea
The shesrflmim Tee
against ooi anatgst
Times cdletftie Tme
The twm nThev
The awlhu Tha
have Hr eva
invite 

In [ ]:
wid_ids_keys = [key for key in wids.keys()]

1


In [ ]:
def prep_data_multiproc(data, num_processes):
    new_data = []
    size = len(data) // num_processes
    for i in range(num_processes ):
        start_idx = i*size
        if i == num_processes - 1:
            new_data.append(data[start_idx:])
        else:
            new_data.append(data[start_idx:start_idx+size])
    return new_data


In [ ]:
tr_data_fn = "../../data/iam/words/ground_truth_train_filtered.txt"
generate_data(train_data, wid_ids_keys)

In [ ]:

test_data_fn = "../../data/iam/words/ground_truth_test_filtered.txt"
generate_data(test_data, wid_ids_keys)

valid_data_fn = "../../data/iam/words/ground_truth_valid_filtered.txt"
generate_data(valid_data, wid_ids_keys)

oov_valid_data_fn = "../../data/iam/words/ground_truth_valid_oov_filtered.txt"
generate_data(oov_valid_data, wid_ids_keys)

oov_test_data_fn = "../../data/iam/words/ground_truth_test_oov_filtered.txt"
generate_data(oov_test_data, wid_ids_keys)

Members otrnD Membes
of nBtarrn ofv
the lnrtnelibo ah
Cabinet nnwereipfg naeiCbt
The tioewtFt eh
process diwpuofn pocses
has leirehub as
been el ben
too trea ot
slow beni laow
for aslovenol oo
Herr  reHhr
Strauss tfhdn Stswusr
and st dan
month crnHh mshnth
he hofmi b
Britain oasCbnhje Britin
for hem fo
being vhspa nebig
an uepniss bn
for hnepnhyns fto
West reiirvoio Wsi
plans  pldan
for ofet ocfr
unified Fh ueifnd
supply lhyknnptrt suplp
in  iun
Nato heyl Ntn
supply dThdH pypslu
The vnAndee we
from erlvn orfm
the laawunawg tea
New uHnehdfl NwE
World nshuno Wlrod
who stac ohw
The thtarnRc T
big lgf be
squeeze oieudd equeezfs
means rtabij heas
that hnae sht
it ilt bts
is io sni
going adrdc gnigo
to oreatfgya toon
be rwnefhta bev
to ctdEiteioe vto
arrange elpticm rranget
loan nwaumthfat lsn
or dr oor
And b d
banks wmllesghem bansn
will ya wel
be tr by
in s hivn
This pnoosn Tps
is hordnty eis
in euhlkaIgrm n
the om htm
Central iennenlPns aentrC
weekly ykiaemnmh weoly
return lMve rehteurn
t

KeyboardInterrupt: 

In [ ]:
# tr_data_fn = "../../data/iam/words/ground_truth_train_filtered.txt"

# with open(tr_data_fn, "w") as tr_f:
#     for tup in train_data:
#         img_path, gt_label, wid = tup[0], tup[1], tup[2]
#         tr_f.write(f"{img_path} {gt_label} {wid}\n")

    

In [ ]:
# test_data_fn = "../../data/iam/words/ground_truth_test_filtered.txt"

# with open(test_data_fn, "w") as test_f:
#     for tup in test_data:
#         img_path, gt_label, wid = tup[0], tup[1], tup[2]
#         test_f.write(f"{img_path} {gt_label} {wid}\n")

# test_data_fn = "../../data/iam/words/ground_truth_test_oov_filtered.txt"

# with open(test_data_fn, "w") as test_f:
#     for tup in oov_test_data:
#         img_path, gt_label, wid = tup[0], tup[1], tup[2]
#         test_f.write(f"{img_path} {gt_label} {wid}\n")

In [ ]:
# valid_data_fn = "../../data/iam/words/ground_truth_valid_filtered.txt"

# with open(valid_data_fn, "w") as valid_f:
#     for tup in valid_data:
#         img_path, gt_label, wid = tup[0], tup[1], tup[2]
#         valid_f.write(f"{img_path} {gt_label} {wid}\n")

# valid_data_fn = "../../data/iam/words/ground_truth_valid_oov_filtered.txt"

# with open(valid_data_fn, "w") as valid_f:
#     for tup in oov_valid_data:
#         img_path, gt_label, wid = tup[0], tup[1], tup[2]
#         valid_f.write(f"{img_path} {gt_label} {wid}\n")

In [ ]:
import numpy as np

file = './test.txt'
count = 0

with open(file, 'r') as f:
    lines = f.read().splitlines()
    for line in lines:
        if len(line.split(' ')) <= 2:
            count += 1
count

4